In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import cv2
import glob
import re
import string
import numpy as np
import pandas as pd 
from tqdm import tqdm
from PIL import Image
import tensorflow.keras
import tensorflow as tf
from skimage import measure
import matplotlib.pyplot as plt
from keras.models import Model
from sklearn import preprocessing
import matplotlib.patches as mpatches 
from tensorflow.keras import layers
from keras import backend as K
import matplotlib.patches as patches
from keras.callbacks import CSVLogger
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2, l1
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from skimage.measure import label, regionprops, regionprops_table
from tensorflow.keras import Sequential, models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.applications import ResNet50, MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten, BatchNormalization, Activation, Input, Add, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D



In [3]:
# load model
model = ResNet50(include_top=False, weights="imagenet", input_shape=(50, 50, 3))
model.summary()

2023-09-19 09:00:01.767954: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 56, 56, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 25, 25, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 25, 25, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 13, 13, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 13, 13, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 13, 13, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 13, 13, 256)  1024       ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 7, 7, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 7, 7, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 7, 7, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 4, 4, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 4, 4, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 4, 4, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 2, 2, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 2, 2, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 2, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [4]:
#define the file paths for the plate annotations, images and corresponding masks
plate_annot = '/4tbint/Plate_annotations_newMoAs_KW/'
plate_path = '/4tbint/Corrected Merged FOVs/Set3/'
mask_path = '/4tbint/Cellpose Masks/Set3/'

#use the glob library to generate lists of image and mask filenames
annotations = sorted([os.path.basename(x) for x in glob.glob(plate_annot+ '/*')])
plates =  sorted([os.path.basename(x) for x in glob.glob(plate_path+ '/*')])
masks = sorted([os.path.basename(x) for x in glob.glob(mask_path+ '/*')])


# When training for DMSO vs treatments (based on cell count range)

In [5]:
image_name_w_paths = [[]]
mask_name_w_paths = []
y_label = []
j=0

for i in range(len(annotations)):
    
    annotation_file = os.path.join(plate_annot, annotations[i])
    annot_file = pd.read_excel(annotation_file)
    labels = ['DMSO', 'Drug']
    annot_file = annot_file.loc[annot_file['Label'].isin(labels)]
    MoA_category = ['EGFR', 'VEGFR', 'PI3K', 'Topoisomerase', 'Mitotic', 'MEK1/2', 'PARP', 'CDK', 'BET', 'HDAC', 'Negative Controls', 'Negative Controls ']
    #MoA_category = ['EGFR','Negative Controls', 'Negative Controls ']
    ann = annot_file.loc[annot_file['Category'].isin(MoA_category)]
    ann = ann.loc[ann["MoA"] != 'cells' ]
    selected_df = ann[(ann['Cancer_Count'] > 600) & (ann['Cancer_Count'] < 2035) | (ann['Label'] == 'DMSO')]
    #selected_df = ann
    
    print(selected_df)
    def well_to_image(well_number):
        row = string.ascii_uppercase.index(well_number[0])+1

        col = int(well_number[1:])

        image_name = f'r{row:02d}c{col:02d}'

        return image_name

    selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)
    
    names = selected_df['Image'].to_list()
    
    image_file = os.path.join(plate_path, plates[i])
    
    mask_file = os.path.join(mask_path, masks[i])

    imgs = sorted([os.path.basename(x) for x in glob.glob(image_file+ '/*.tiff')])

    msks = sorted([os.path.basename(x) for x in glob.glob(mask_file+ '/*.tiff')])
    
    for img in range(0, len(imgs), 3):
        imagename_ch1 = imgs[img]
        imagename_ch2 = imgs[img+1]
        imagename_ch4 = imgs[img+2]
        
        prefix_im = imagename_ch1.split('ch')[0]
        
        if prefix_im in names:
            
            im_ch1 = os.path.join(image_file, imagename_ch1)
            im_ch2 = os.path.join(image_file, imagename_ch2)
            im_ch4 = os.path.join(image_file, imagename_ch4)
            
            image_name_w_paths[j].append(im_ch1)
            image_name_w_paths[j].append(im_ch2)
            image_name_w_paths[j].append(im_ch4)
            image_name_w_paths.append([])
            
            cat = selected_df.loc[selected_df['Image'] == prefix_im, 'Label'].values[0]
            
            y_label.append(cat)   
            
            j=j+1
            
    for maskname in msks:
        prefix_msk = maskname.split('ch')[0]
        if prefix_msk in names:
            msk = os.path.join(mask_file, maskname)
            mask_name_w_paths.append(msk)
            
   
                 
image_names_w_paths = list(filter(None, image_name_w_paths))
X = image_names_w_paths    
X_cellpose = mask_name_w_paths
Y = y_label

print(len(X))
print(len(X_cellpose))
print(len(Y))

    Well_number    Compound  \
26           B3  Vorinostat   
32           B9        DMSO   
59          C12        DMSO   
65          C18  Paclitaxel   
67          C20        DMSO   
76           D5        DMSO   
110         E15        DMSO   
178         H11   Amsacrine   
186         H19        DMSO   
187         H20  Epirubicin   
199          I8        DMSO   
211         I20  Epirubicin   
250         K11       SN-38   
254         K15    Eribulin   
277         L14   Topotecan   
280         L17        DMSO   
301         M14   Topotecan   
318          N7        DMSO   
325         N14   Topotecan   
331         N20  Vinflunine   
337          O2    Olaparib   
341          O6   Rucaparib   
348         O13        DMSO   
355         O20  Vinflunine   
361          P2    Olaparib   
362          P3        DMSO   
365          P6   Rucaparib   
381         P22        DMSO   

                                                   MoA Concentration  \
26                          

/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)
/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)
/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

    Well_number     Compound                                        MoA  \
12          A13    PD0325901                           MEK1/2 inhibitor   
18          A19    Alpelisib                        PI3Kalpha inhibitor   
19          A20    Motesanib           VEGFR, PDGFR, Ret, Kit inhibitor   
31           B8    Milciclib                             CDK2 inhibitor   
32           B9         DMSO                                       DMSO   
35          B12    ENMD-2076                pan-Aurora, VEGFR inhibitor   
51           C4      SNS-032                              CDK inhibitor   
59          C12         DMSO                                       DMSO   
67          C20         DMSO                                       DMSO   
76           D5         DMSO                                       DMSO   
83          D12    ENMD-2076                pan-Aurora, VEGFR inhibitor   
110         E15         DMSO                                       DMSO   
123          F4   Selicic

/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)
/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)
/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

    Well_number      Compound                         MoA Concentration  \
2            A3   Talazoparib           PARP1/2 inhibitor          1000   
8            A9    Givinostat              HDAC inhibitor          1000   
11          A12  Rocilinostat  HDAC-6 selective inhibitor         10000   
20          A21         dBET1        BET-targeting PROTAC         10000   
21          A22         PFI-1        BET family inhibitor         30000   
..          ...           ...                         ...           ...   
373         P14  Tubastatin A             HDAC6 inhibitor         10000   
375         P16       RGFP966             HDAC3 inhibitor         10000   
379         P20           JQ1        BET family inhibitor         10000   
381         P22          DMSO                        DMSO             0   
382         P23       ARV-825        BET-targeting PROTAC           300   

     Cancer_Count Label           Category  
2             709  Drug               PARP  
8        

/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)
/tmp/ipykernel_140701/3161304548.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Image'] = selected_df['Well_number'].apply(well_to_image)


In [6]:
label_encoder = preprocessing.LabelEncoder()
Y_Y = label_encoder.fit_transform(Y)


In [7]:
X_train, X_val, Y_train, Y_val, X_train_cellpose, X_val_cellpose = train_test_split(X, Y_Y, X_cellpose, test_size=0.3, random_state=42) 

print("The length of training dataset: {}".format(len(X_train)))
print("The length of validation dataset: {}".format(len(X_val)))

The length of training dataset: 202
The length of validation dataset: 87


In [8]:
print(np.unique(Y))
print(np.unique(Y_Y))

['DMSO' 'Drug']
[0 1]


In [9]:
from collections import Counter
Counter(Y_Y)

Counter({1: 193, 0: 96})

In [10]:
single_cell_bbox = []
single_cell_bbox_label = []

for i in range(len(X_train)):
    
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(X_train[i][0])
    ch2_img = Image.open(X_train[i][1])
    ch4_img = Image.open(X_train[i][2])
    cellpose_img = Image.open(X_train_cellpose[i])
    
    ch1_img_arr = np.array(ch1_img)
    ch2_img_arr = np.array(ch2_img)
    ch4_img_arr = np.array(ch4_img)
    cellpose_img_array = np.array(cellpose_img)

    #normalize the pixel values of the image arrays to be between 0 and 65535.
    ch1_img_arr = ch1_img_arr/(ch1_img_arr.max()/65535.0)
    ch2_img_arr = ch2_img_arr/(ch2_img_arr.max()/65535.0)
    ch4_img_arr = ch4_img_arr/(ch4_img_arr.max()/65535.0)
    
    
    labels = label(cellpose_img_array)
    regions = measure.regionprops(labels)
    
    
    for props in regions:
        y0, x0 = props.centroid
        y = int(round(y0))
        x = int(round(x0))
        box_ch1 = ch1_img_arr[x-25:x+25,y-25:y+25]
        box_ch2 = ch2_img_arr[x-25:x+25,y-25:y+25]
        box_ch4 = ch4_img_arr[x-25:x+25,y-25:y+25]

        if (box_ch1.shape == (50,50)):
            coordinate = x, y
            singlecell = np.stack((box_ch1, box_ch2, box_ch4), axis=-1)
            singlecell = (singlecell/65535)*255
            singlecell = np.uint8(singlecell)
            reshaped_singlecell = singlecell.reshape((50,50,3))
            single_cell_bbox.append(reshaped_singlecell)
            single_cell_bbox_label.append(Y_train[i])
x_train = np.asarray(single_cell_bbox)
y_train = np.asarray(single_cell_bbox_label)       

In [11]:
single_cell_bbox = []
single_cell_bbox_label = []

for i in range(len(X_val)):
    #print(X_train[i][0])
    #print(X_train_cellpose[i])
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(X_val[i][0])
    ch2_img = Image.open(X_val[i][1])
    ch4_img = Image.open(X_val[i][2])
    cellpose_img = Image.open(X_val_cellpose[i])
    
    ch1_img_arr = np.array(ch1_img)
    ch2_img_arr = np.array(ch2_img)
    ch4_img_arr = np.array(ch4_img)
    cellpose_img_array = np.array(cellpose_img)

    #normalize the pixel values of the image arrays to be between 0 and 65535.
    ch1_img_arr = ch1_img_arr/(ch1_img_arr.max()/65535.0)
    ch2_img_arr = ch2_img_arr/(ch2_img_arr.max()/65535.0)
    ch4_img_arr = ch4_img_arr/(ch4_img_arr.max()/65535.0)
    
    
    labels = label(cellpose_img_array)
    regions = measure.regionprops(labels)
    
    
    for props in regions:
        y0, x0 = props.centroid
        y = int(round(y0))
        x = int(round(x0))
        box_ch1 = ch1_img_arr[x-25:x+25,y-25:y+25]
        box_ch2 = ch2_img_arr[x-25:x+25,y-25:y+25]
        box_ch4 = ch4_img_arr[x-25:x+25,y-25:y+25]

        if (box_ch1.shape == (50,50)):
            coordinate = x, y
            singlecell = np.stack((box_ch1, box_ch2, box_ch4), axis=-1)
            singlecell = (singlecell/65535)*255
            singlecell = np.uint8(singlecell)
            reshaped_singlecell = singlecell.reshape((50,50,3))
            single_cell_bbox.append(reshaped_singlecell)
            single_cell_bbox_label.append(Y_val[i])
x_val = np.asarray(single_cell_bbox)
y_val = np.asarray(single_cell_bbox_label)       

In [12]:
#Unfreeze some layers and train

input_shape=(50, 50, 3)

conv_base = ResNet50(weights='imagenet',include_top=False,input_shape=input_shape)

print(conv_base.summary())

for layer in conv_base.layers:
    layer.trainable = False
print('Last block of the conv_base is not trainable')


for layer in conv_base.layers[103:]:
    layer.trainable = True
print('Last block of the conv_base is not trainable')


for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)

'''
#Train only top classification and not cnn layers
input_shape=(50, 50, 3)

conv_base = ResNet50(weights='imagenet',include_top=False,input_shape=input_shape)

print(conv_base.summary())

for layer in conv_base.layers:
    layer.trainable = False
print('Last block of the conv_base is not trainable')
'''

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 56, 56, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 25, 25, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 25, 25, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 13, 13, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 13, 13, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 13, 13, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 13, 13, 256)  1024       ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 7, 7, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 7, 7, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 7, 7, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 4, 4, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 4, 4, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 4, 4, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 4, 4, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 4, 4, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 2, 2, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 2, 2, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 2, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

"\n#Train only top classification and not cnn layers\ninput_shape=(50, 50, 3)\n\nconv_base = ResNet50(weights='imagenet',include_top=False,input_shape=input_shape)\n\nprint(conv_base.summary())\n\nfor layer in conv_base.layers:\n    layer.trainable = False\nprint('Last block of the conv_base is not trainable')\n"

In [13]:
num_classes = 1
'''
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dense(num_classes, activation='sigmoid')) 
print(model.summary())
'''

'''
#******************************Model with top layer training including dropout******************************
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
#model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dense(num_classes, activation='sigmoid')) 
print(model.summary())
'''

#******************************Model with top layer training with unfreezed layers******************************
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation= 'relu', kernel_regularizer=l1(0.01)))
model.add(Dropout(0.1))
model.add(layers.Dense(512, activation='sigmoid', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.1))
#model.add(BatchNormalization())
model.add(layers.Dense(num_classes, activation='sigmoid')) 
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                        

In [14]:
train_data_gen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input,
        rotation_range=10, 
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        vertical_flip=True, # vertical flip                           
        shear_range=0.2) # shear distortion along axis


train_generator = train_data_gen.flow(x_train,y_train,batch_size=32)

In [15]:
#OPT = tf.keras.optimizers.SGD(learning_rate=0.0001) 

#Exponential learning rate schedular. 
initial_learning_rate = 0.001
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)

OPT = Adam(learning_rate=lr_scheduler)

model.compile(optimizer=OPT,loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
csv_logger = tf.keras.callbacks.CSVLogger("/home/gauss/Desktop/Training_KW/KW_DMSO_DRUG4/TrainingLog.csv", separator=",", append=True)

#history = model.fit(x_train, y_train, batch_size=16, epochs=100, validation_data=(x_val, y_val),callbacks=[csv_logger], verbose=1)

history = model.fit_generator(train_generator,epochs=50, validation_data=(x_val, y_val),callbacks=[csv_logger], verbose=1)


Epoch 1/50


/tmp/ipykernel_140701/4154541231.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,epochs=50, validation_data=(x_val, y_val),callbacks=[csv_logger], verbose=1)
2023-09-19 09:04:13.743175: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-09-19 09:04:23.169606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-09-19 09:04:25.560782: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f9354080ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-19 09:04:25.560831: I tensorflow/compiler/xla/serv

10446/10446 [==============================] - 669s 62ms/step - loss: 3.6759 - accuracy: 0.6120 - val_loss: 2.7853 - val_accuracy: 0.5075
Epoch 2/50
10446/10446 [==============================] - 653s 63ms/step - loss: 2.6984 - accuracy: 0.6159 - val_loss: 2.5320 - val_accuracy: 0.5075
Epoch 3/50
10446/10446 [==============================] - 653s 63ms/step - loss: 2.5266 - accuracy: 0.6169 - val_loss: 2.4871 - val_accuracy: 0.5075
Epoch 4/50
10446/10446 [==============================] - 636s 61ms/step - loss: 2.3607 - accuracy: 0.6167 - val_loss: 2.2886 - val_accuracy: 0.5075
Epoch 5/50
10446/10446 [==============================] - 642s 61ms/step - loss: 2.1118 - accuracy: 0.6217 - val_loss: 2.0355 - val_accuracy: 0.5075
Epoch 6/50
10446/10446 [==============================] - 637s 61ms/step - loss: 1.8890 - accuracy: 0.6261 - val_loss: 1.9146 - val_accuracy: 0.5078
Epoch 7/50
10446/10446 [==============================] - 644s 62ms/step - loss: 1.7558 - accuracy: 0.6296 - val_loss

In [17]:
#!mkdir -p saved_model
model.save("/home/gauss/Desktop/Training_KW/KW_DMSO_DRUG4/Trained_Model")


In [18]:
#summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("/home/gauss/Desktop/Training_KW/KW_DMSO_DRUG4/Accuracy.png")
plt.show()

In [ ]:
#summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("/home/gauss/Desktop/Training_KW/KW_DMSO_DRUG4/Loss.png")
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/home/gauss/Desktop/Training_KW/KW_DMSO_DRUG4/Trained_Model")
model.summary()